In [6]:
!pip install statsmodels

You should consider upgrading via the 'pip install --upgrade pip' command.


In [8]:
pip install scikit-learn

     |████████████████████████████████| 13.4MB 6.4MB/s eta 0:00:01
     |████████████████████████████████| 307kB 85.4MB/s eta 0:00:01
You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [9]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error, r2_score



In [11]:
pip install matplotlib


     |████████████████████████████████| 8.3MB 7.1MB/s eta 0:00:01
     |████████████████████████████████| 1.6MB 62.2MB/s eta 0:00:01
     |████████████████████████████████| 4.6MB 64.1MB/s eta 0:00:01
     |████████████████████████████████| 112kB 95.6MB/s eta 0:00:01
     |████████████████████████████████| 327kB 86.0MB/s eta 0:00:01
     |████████████████████████████████| 4.4MB 81.1MB/s eta 0:00:01
You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [13]:
!pip install fosforml

     |████████████████████████████████| 51kB 4.8MB/s eta 0:00:011
     |████████████████████████████████| 1.9MB 8.1MB/s eta 0:00:01
     |████████████████████████████████| 10.9MB 71.5MB/s eta 0:00:01
     |████████████████████████████████| 133kB 109.0MB/s eta 0:00:01
     |████████████████████████████████| 81kB 27.1MB/s eta 0:00:01
     |████████████████████████████████| 200.3MB 98kB/s /s eta 0:00:01
     |████████████████████████████████| 1.3MB 70.2MB/s eta 0:00:01
     |████████████████████████████████| 98.7MB 256kB/s  eta 0:00:01
     |████████████████████████████████| 2.5MB 41.5MB/s eta 0:00:01
     |████████████████████████████████| 39.9MB 72.1MB/s eta 0:00:01
     |████████████████████████████████| 174kB 103.5MB/s eta 0:00:01
     |████████████████████████████████| 61kB 22.5MB/s eta 0:00:01
     |████████████████████████████████| 51kB 22.1MB/s eta 0:00:01
     |████████████████████████████████| 19.1MB 87.6MB/s eta 0:00:01
     |████████████████████████████████| 51kB 20.5MB/s eta 

In [14]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt

# Assuming 'df' is the DataFrame that contains your data
# df = pd.read_csv('your_data.csv')

from fosforml.model_manager.snowflakesession import get_session
my_session = get_session()
 
table_name = '"MASTER_DATA"'
 
sf_df = my_session.sql("select * from {}".format(table_name))
df = sf_df.to_pandas()


# Convert 'ORDER_DATE' to datetime
df['ORDER_DATE'] = pd.to_datetime(df['ORDER_DATE'])

# Sort data by 'ORDER_DATE' to maintain time series order
df = df.sort_values(by='ORDER_DATE')

# Function to create multiple lag features for ORDER_QTY
def create_lags(df, target_column, num_lags):
    for lag in range(1, num_lags + 1):
        df[f'Lag_{lag}'] = df[target_column].shift(lag)
    return df

# Define a function that applies the prediction process for each product
def forecast_for_product(product_df, num_lags=6, forecast_periods=25):
    # Create lag features for ORDER_QTY
    product_df = create_lags(product_df, 'ORDER_QTY', num_lags)
    
    # Drop rows with NaN values (due to shifting from lag features)
    product_df = product_df.dropna()
    
    # Define the features (Lag_1, Lag_2, ..., Lag_n, UNIT_PRICE, LEAD_TIME_IN_WEEKS) and target (ORDER_QTY)
    lag_columns = [f'Lag_{lag}' for lag in range(1, num_lags + 1)]
    feature_columns = lag_columns + ['UNIT_PRICE', 'LEAD_TIME_IN_WEEKS']
    
    # Define X (features) and y (target)
    X = product_df[feature_columns]
    y = product_df['ORDER_QTY']
    
    # Split the data into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
    
    # Initialize the Linear Regression model
    model = LinearRegression()
    
    # Fit the model to the training data
    model.fit(X_train, y_train)
    
    # Forecast for the next 25 months
    # Create a new DataFrame for future dates
    future_dates = pd.date_range(product_df['ORDER_DATE'].max(), periods=forecast_periods, freq='M')
    
    # Initialize the last known lag values, UNIT_PRICE, and LEAD_TIME_IN_WEEKS
    last_lags = list(product_df[lag_columns].iloc[-1])
    last_unit_price = product_df['UNIT_PRICE'].iloc[-1]
    last_lead_time = product_df['LEAD_TIME_IN_WEEKS'].iloc[-1]
    
    # Create an empty list to store predictions
    future_preds = []
    
    for i in range(forecast_periods):
        # Prepare input features for the next prediction, including lag values, UNIT_PRICE, and LEAD_TIME_IN_WEEKS
        future_X = np.array([last_lags + [last_unit_price, last_lead_time]])
        
        # Predict the next ORDER_QTY
        future_pred = model.predict(future_X)[0]
        
        # Append the prediction to the list
        future_preds.append(future_pred)
        
        # Update lag values for the next iteration
        last_lags = [future_pred] + last_lags[:-1]  # Shift the lags with the new prediction

    # Create a DataFrame for this product's future ORDER_QTY predictions
    future_forecast_df = pd.DataFrame({
        'ORDER_DATE': future_dates,
        'Predicted_ORDER_QTY': future_preds,
        'PRODUCT_ID': product_df['PRODUCT_ID'].iloc[0]  # Add the PRODUCT_ID to each row
    })
    
    return future_forecast_df

# Initialize an empty DataFrame to store all product forecasts
all_products_forecast_df = pd.DataFrame()

# Loop over each product group by 'PRODUCT_ID'
for product_id, product_df in df.groupby('PRODUCT_ID'):
    # Apply the forecast function for each product
    product_forecast_df = forecast_for_product(product_df)
    
    # Append the forecast for this product to the overall forecast DataFrame
    all_products_forecast_df = pd.concat([all_products_forecast_df, product_forecast_df], ignore_index=True)

# Rename the columns
all_products_forecast_df.rename(columns={'Predicted_ORDER_QTY': 'ORDER_QTY'}, inplace=True)

# Display the combined forecast DataFrame for all products
print(all_products_forecast_df)

# Optionally, save the resulting DataFrame to a CSV file
# all_products_forecast_df.to_csv('all_products_order_qty_forecast.csv', index=False)

     ORDER_DATE   ORDER_QTY   PRODUCT_ID
0    2024-08-31  139.262531  Product_001
1    2024-09-30  166.985132  Product_001
2    2024-10-31  192.463623  Product_001
3    2024-11-30  216.339747  Product_001
4    2024-12-31  237.965688  Product_001
...         ...         ...          ...
1095 2026-04-30  515.715568  Product_100
1096 2026-05-31  515.865625  Product_100
1097 2026-06-30  515.977730  Product_100
1098 2026-07-31  515.967549  Product_100
1099 2026-08-31  515.909618  Product_100

[1100 rows x 3 columns]


In [20]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt

# Assuming 'df' is the DataFrame that contains your data
# df = pd.read_csv('your_data.csv')

from fosforml.model_manager.snowflakesession import get_session
my_session = get_session()
 
table_name = '"MASTER_DATA"'
 
sf_df = my_session.sql("select * from {}".format(table_name))
df = sf_df.to_pandas()


# Convert 'ORDER_DATE' to datetime
df['ORDER_DATE'] = pd.to_datetime(df['ORDER_DATE'])

# Sort data by 'ORDER_DATE' to maintain time series order
df = df.sort_values(by='ORDER_DATE')

# Function to create multiple lag features for ORDER_QTY
def create_lags(df, target_column, num_lags):
    for lag in range(1, num_lags + 1):
        df[f'Lag_{lag}'] = df[target_column].shift(lag)
    return df

# Define a function that applies the prediction process for each product, along with RMSE and R² calculation
def forecast_for_product(product_df, num_lags=6, forecast_periods=25):
    # Create lag features for ORDER_QTY
    product_df = create_lags(product_df, 'ORDER_QTY', num_lags)
    
    # Drop rows with NaN values (due to shifting from lag features)
    product_df = product_df.dropna()
    
    # Define the features (Lag_1, Lag_2, ..., Lag_n, UNIT_PRICE, LEAD_TIME_IN_WEEKS) and target (ORDER_QTY)
    lag_columns = [f'Lag_{lag}' for lag in range(1, num_lags + 1)]
    feature_columns = lag_columns + ['UNIT_PRICE', 'LEAD_TIME_IN_WEEKS','ORDER_DATE']
    
    # Define X (features) and y (target)
    X = product_df[feature_columns]
    y = product_df['ORDER_QTY']
    
    # Split the data into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
    
    # Initialize the Linear Regression model
    model = LinearRegression()
    
    # Fit the model to the training data
    model.fit(X_train, y_train)
    
    # Make predictions on the test data
    y_pred = model.predict(X_test)
    
    # Calculate RMSE
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    
    # Calculate R²
    r2 = r2_score(y_test, y_pred)
    
    # Forecast for the next 25 months
    # Create a new DataFrame for future dates
    future_dates = pd.date_range(product_df['ORDER_DATE'].max(), periods=forecast_periods, freq='M')
    
    # Initialize the last known lag values, UNIT_PRICE, and LEAD_TIME_IN_WEEKS
    last_lags = list(product_df[lag_columns].iloc[-1])
    last_unit_price = product_df['UNIT_PRICE'].iloc[-1]
    last_lead_time = product_df['LEAD_TIME_IN_WEEKS'].iloc[-1]
    
    # Create an empty list to store predictions
    future_preds = []
    
    for i in range(forecast_periods):
        # Prepare input features for the next prediction, including lag values, UNIT_PRICE, and LEAD_TIME_IN_WEEKS
        future_X = np.array([last_lags + [last_unit_price, last_lead_time]])
        
        # Predict the next ORDER_QTY
        future_pred = model.predict(future_X)[0]
        
        # Append the prediction to the list
        future_preds.append(future_pred)
        
        # Update lag values for the next iteration
        last_lags = [future_pred] + last_lags[:-1]  # Shift the lags with the new prediction

    # Create a DataFrame for this product's future ORDER_QTY predictions

    print(f"PRODUCT_ID: {product_df['PRODUCT_ID']}, RMSE: {rmse}, R2: {r2}")
    future_forecast_df = pd.DataFrame({
        'ORDER_DATE': future_dates,
        'Predicted_ORDER_QTY': future_preds,
        'PRODUCT_ID': product_df['PRODUCT_ID'].iloc[0],  # Add the PRODUCT_ID to each row
        #'RMSE': rmse,  # Include the RMSE value for this product
        #'R2': r2       # Include the R² value for this product
    })
    
    return future_forecast_df

# Initialize an empty DataFrame to store all product forecasts
all_products_forecast_df = pd.DataFrame()

# Loop over each product group by 'PRODUCT_ID'
for product_id, product_df in df.groupby('PRODUCT_ID'):
    # Apply the forecast function for each product
    product_forecast_df = forecast_for_product(product_df)
    
    # Append the forecast for this product to the overall forecast DataFrame
    all_products_forecast_df = pd.concat([all_products_forecast_df, product_forecast_df], ignore_index=True)

# Rename the columns
all_products_forecast_df.rename(columns={'Predicted_ORDER_QTY': 'ORDER_QTY'}, inplace=True)

# Display the combined forecast DataFrame for all products
print(all_products_forecast_df)

# Optionally, save the resulting DataFrame to a CSV file
# all_products_forecast_df.to_csv('all_products_order_qty_forecast_with_rmse_r2.csv', index=False)

PRODUCT_ID: 22         Product_001
21         Product_001
20         Product_001
19         Product_001
18         Product_001
              ...     
4206542    Product_001
4206543    Product_001
4206544    Product_001
4206545    Product_001
4206531    Product_001
Name: PRODUCT_ID, Length: 642688, dtype: object, RMSE: 134.6776144283198, R2: 0.7844648646338515
PRODUCT_ID: 3423       Product_002
3424       Product_002
3425       Product_002
3427       Product_002
3429       Product_002
              ...     
4211236    Product_002
4211237    Product_002
4211238    Product_002
4211239    Product_002
4211241    Product_002
Name: PRODUCT_ID, Length: 275030, dtype: object, RMSE: 145.4870708968556, R2: 0.7475631645692312
PRODUCT_ID: 3766       Product_003
3765       Product_003
3764       Product_003
3763       Product_003
3762       Product_003
              ...     
4212715    Product_003
4212714    Product_003
4212713    Product_003
4212712    Product_003
4212724    Product_003
Name: PRODU

In [26]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt

# Assuming 'df' is the DataFrame that contains your data
# df = pd.read_csv('your_data.csv')

from fosforml.model_manager.snowflakesession import get_session
my_session = get_session()
 
table_name = '"MASTER_DATA"'
 
sf_df = my_session.sql("select * from {}".format(table_name))
df = sf_df.to_pandas()


# Convert 'ORDER_DATE' to datetime
df['ORDER_DATE'] = pd.to_datetime(df['ORDER_DATE'])

# Sort data by 'ORDER_DATE' to maintain time series order
df = df.sort_values(by='ORDER_DATE')

# Function to create multiple lag features for ORDER_QTY
def create_lags(df, target_column, num_lags):
    for lag in range(1, num_lags + 1):
        df[f'Lag_{lag}'] = df[target_column].shift(lag)
    return df

# Define a function that applies the prediction process for each product, along with RMSE and R² calculation
def forecast_for_product(product_df, num_lags=6, forecast_periods=25):
    # Create lag features for ORDER_QTY
    product_df = create_lags(product_df, 'ORDER_QTY', num_lags)
    
    # Drop rows with NaN values (due to shifting from lag features)
    product_df = product_df.dropna()

    # Convert 'ORDER_DATE' to an ordinal number to use as a feature
    product_df['ORDER_DATE_ORDINAL'] = product_df['ORDER_DATE'].apply(lambda x: x.toordinal())
    
    # Define the features (Lag_1, Lag_2, ..., Lag_n, UNIT_PRICE, LEAD_TIME_IN_WEEKS, ORDER_DATE_ORDINAL)
    lag_columns = [f'Lag_{lag}' for lag in range(1, num_lags + 1)]
    feature_columns = lag_columns + ['UNIT_PRICE', 'LEAD_TIME_IN_WEEKS', 'ORDER_DATE_ORDINAL']
    
    # Define X (features) and y (target)
    X = product_df[feature_columns]
    y = product_df['ORDER_QTY']
    
    # Split the data into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
    
    # Initialize the Linear Regression model
    model = LinearRegression()
    
    # Fit the model to the training data
    model.fit(X_train, y_train)
    
    # Make predictions on the test data
    y_pred = model.predict(X_test)
    
    # Calculate RMSE
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    
    # Calculate R²
    r2 = r2_score(y_test, y_pred)
    
    # Forecast for the next 25 months
    # Create a new DataFrame for future dates
    future_dates = pd.date_range(product_df['ORDER_DATE'].max(), periods=forecast_periods, freq='M')
    
    # Initialize the last known lag values, UNIT_PRICE, LEAD_TIME_IN_WEEKS, and ORDER_DATE_ORDINAL
    last_lags = list(product_df[lag_columns].iloc[-1])
    last_unit_price = product_df['UNIT_PRICE'].iloc[-1]
    last_lead_time = product_df['LEAD_TIME_IN_WEEKS'].iloc[-1]
    last_order_date_ordinal = product_df['ORDER_DATE_ORDINAL'].iloc[-1]
    
    # Create an empty list to store predictions
    future_preds = []
    
    for i in range(forecast_periods):
        # Increment the ORDER_DATE_ORDINAL for the next future date
        last_order_date_ordinal += 30  # Assuming average month is ~30 days
        
        # Prepare input features for the next prediction, including lag values, UNIT_PRICE, LEAD_TIME_IN_WEEKS, and ORDER_DATE_ORDINAL
        future_X = np.array([last_lags + [last_unit_price, last_lead_time, last_order_date_ordinal]])
        
        # Predict the next ORDER_QTY
        future_pred = model.predict(future_X)[0]
        
        # Append the prediction to the list
        future_preds.append(future_pred)
        
        # Update lag values for the next iteration
        last_lags = [future_pred] + last_lags[:-1]  # Shift the lags with the new prediction

    # Create a DataFrame for this product's future ORDER_QTY predictions
    print(f"RMSE: {rmse}, R2: {r2}")
    future_forecast_df = pd.DataFrame({
        'ORDER_DATE': future_dates,
        'Predicted_ORDER_QTY': future_preds,
        'PRODUCT_ID': product_df['PRODUCT_ID'].iloc[0],  # Add the PRODUCT_ID to each row
        #'RMSE': rmse,  # Include the RMSE value for this product
        #'R2': r2       # Include the R² value for this product
    })
    
    return future_forecast_df

# Initialize an empty DataFrame to store all product forecasts
all_products_forecast_df = pd.DataFrame()

# Loop over each product group by 'PRODUCT_ID'
for product_id, product_df in df.groupby('PRODUCT_ID'):
    # Apply the forecast function for each product
    product_forecast_df = forecast_for_product(product_df)
    
    # Append the forecast for this product to the overall forecast DataFrame
    all_products_forecast_df = pd.concat([all_products_forecast_df, product_forecast_df], ignore_index=True)

# Rename the columns
all_products_forecast_df.rename(columns={'Predicted_ORDER_QTY': 'ORDER_QTY'}, inplace=True)

# Display the combined forecast DataFrame for all products
print(all_products_forecast_df)

# Optionally, save the resulting DataFrame to a CSV file
# all_products_forecast_df.to_csv('all_products_order_qty_forecast_with_rmse_r2_and_order_date.csv', index=False)

RMSE: 134.67825527928488, R2: 0.7844628134209795
RMSE: 145.49282039008438, R2: 0.7475432121082239
RMSE: 122.17798496652166, R2: 0.8237717078996525
RMSE: 125.19898186616601, R2: 0.8114195634842362
RMSE: 105.84465277679021, R2: 0.8556395167429396
RMSE: 141.88266101943006, R2: 0.7565005495303758
RMSE: 147.1075810258403, R2: 0.7305709018287487
RMSE: 120.43251167376745, R2: 0.8288652760532603
RMSE: 129.21352834648786, R2: 0.7956449277674785
RMSE: 131.46137509772637, R2: 0.7927057345863692
RMSE: 125.93745213660289, R2: 0.8061097372246434
RMSE: 155.02594487740905, R2: 0.7102045976755728
RMSE: 151.37529263503552, R2: 0.713390638315427
RMSE: 174.86904867018276, R2: 0.635477619259976
RMSE: 189.1768329312456, R2: 0.5746367132112963
RMSE: 144.74371635021845, R2: 0.750286623786495
RMSE: 182.0639547805628, R2: 0.5897084586708683
RMSE: 196.93756765087582, R2: 0.4988695904981212
RMSE: 191.3601602547579, R2: 0.5687941969459089
RMSE: 252.7810557454304, R2: 0.2572303090039567
RMSE: 169.7136607545755, R2:

In [34]:
# Let's convert 'ORDER_MONTH' to datetime and proceed with forecasting for each 'PRODUCT_ID'
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt


from fosforml.model_manager.snowflakesession import get_session
my_session = get_session()
 
table_name = '"AGG_QTY_MASTER_DATA"'
 
sf_df = my_session.sql("select * from {}".format(table_name))
df = sf_df.to_pandas()



df['ORDER_MONTH'] = pd.to_datetime(df['ORDER_MONTH'])

print(f"{df['ORDER_MONTH']}")

# Sort the DataFrame by 'ORDER_MONTH' to maintain the time series order
df = df.sort_values(by=['PRODUCT_ID', 'ORDER_MONTH'])

# Function to create multiple lag features for ORDER_QTY
def create_lags(df, target_column, num_lags):
    for lag in range(1, num_lags + 1):
        df[f'Lag_{lag}'] = df[target_column].shift(lag)
    return df

# Define a function that applies the prediction process for each product, including RMSE and R² calculation
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

def forecast_for_product(product_df, num_lags=6, forecast_periods=25):
    # Create lag features for ORDER_QTY
    product_df = create_lags(product_df, 'ORDER_QTY', num_lags)
    
    # Drop rows with NaN values (due to shifting from lag features)
    product_df = product_df.dropna()

    # Convert 'ORDER_MONTH' to an ordinal number to use as a feature
    product_df['ORDER_MONTH_ORDINAL'] = product_df['ORDER_MONTH'].apply(lambda x: x.toordinal())
    
    # Define the features (Lag_1, Lag_2, ..., Lag_n, UNIT_PRICE, LEAD_TIME_IN_WEEKS, ORDER_MONTH_ORDINAL)
    lag_columns = [f'Lag_{lag}' for lag in range(1, num_lags + 1)]
    feature_columns = lag_columns + ['UNIT_PRICE', 'LEAD_TIME_IN_WEEKS', 'ORDER_MONTH_ORDINAL']
    
    # Define X (features) and y (target)
    X = product_df[feature_columns]
    y = product_df['ORDER_QTY']
    
    # Split the data into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
    
    # Initialize the Linear Regression model
    model = LinearRegression()
    
    # Fit the model to the training data
    model.fit(X_train, y_train)
    
    # Make predictions on the test data
    y_pred = model.predict(X_test)
    
    # Calculate RMSE
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    
    # Calculate R²
    r2 = r2_score(y_test, y_pred)
    
    # Forecast for the next 25 months
    # Create a new DataFrame for future months
    future_months = pd.date_range(product_df['ORDER_MONTH'].max(), periods=forecast_periods, freq='M')
    
    # Initialize the last known lag values, UNIT_PRICE, LEAD_TIME_IN_WEEKS, and ORDER_MONTH_ORDINAL
    last_lags = list(product_df[lag_columns].iloc[-1])
    last_unit_price = product_df['UNIT_PRICE'].iloc[-1]
    last_lead_time = product_df['LEAD_TIME_IN_WEEKS'].iloc[-1]
    last_order_month_ordinal = product_df['ORDER_MONTH_ORDINAL'].iloc[-1]
    
    # Create an empty list to store predictions
    future_preds = []
    
    for i in range(forecast_periods):
        # Increment the ORDER_MONTH_ORDINAL for the next future month
        last_order_month_ordinal += 30  # Assuming average month is ~30 days
        
        # Prepare input features for the next prediction, including lag values, UNIT_PRICE, LEAD_TIME_IN_WEEKS, and ORDER_MONTH_ORDINAL
        future_X = np.array([last_lags + [last_unit_price, last_lead_time, last_order_month_ordinal]])
        
        # Predict the next ORDER_QTY
        future_pred = model.predict(future_X)[0]
        
        # Append the prediction to the list
        future_preds.append(future_pred)
        
        # Update lag values for the next iteration
        last_lags = [future_pred] + last_lags[:-1]  # Shift the lags with the new prediction
    print(f"RMSE: {rmse}, R2: {r2}")
    # Create a DataFrame for this product's future ORDER_QTY predictions
    future_forecast_df = pd.DataFrame({
        'ORDER_MONTH': future_months,
        'Predicted_ORDER_QTY': future_preds,
        'PRODUCT_ID': product_df['PRODUCT_ID'].iloc[0],  # Add the PRODUCT_ID to each row
       # 'RMSE': rmse,  # Include the RMSE value for this product
       # 'R2': r2       # Include the R² value for this product
    })
    
    return future_forecast_df

# Initialize an empty DataFrame to store all product forecasts
all_products_forecast_df = pd.DataFrame()

# Loop over each product group by 'PRODUCT_ID'
for product_id, product_df in df.groupby('PRODUCT_ID'):
    # Apply the forecast function for each product
    product_forecast_df = forecast_for_product(product_df)
    
    # Append the forecast for this product to the overall forecast DataFrame
    all_products_forecast_df = pd.concat([all_products_forecast_df, product_forecast_df], ignore_index=True)

# Rename the columns
all_products_forecast_df.rename(columns={'Predicted_ORDER_QTY': 'ORDER_QTY'}, inplace=True)

all_products_forecast_df

0       2019-01-01
1       2019-02-01
2       2019-03-01
3       2019-04-01
4       2019-05-01
           ...    
38755   2024-04-01
38756   2024-05-01
38757   2024-06-01
38758   2024-07-01
38759   2024-08-01
Name: ORDER_MONTH, Length: 38760, dtype: datetime64[ns]
RMSE: 3687.002932116978, R2: 0.9435212482278796
RMSE: 2837.5596130015138, R2: 0.9266556444583933
RMSE: 895.8436596669645, R2: 0.972208352852917
RMSE: 1603.7940803740207, R2: 0.9684789519351817
RMSE: 1096.8528164770764, R2: 0.9614890672051816
RMSE: 3173.3373242387925, R2: 0.9298031243384767
RMSE: 3244.63489239995, R2: 0.9218043518056516
RMSE: 2276.403373665819, R2: 0.965580365908683
RMSE: 3037.116577401367, R2: 0.9678371693378719
RMSE: 2178.1151311147137, R2: 0.9460983484897659
RMSE: 1010.3569650544092, R2: 0.9164159062122962
RMSE: 2005.4469041369653, R2: 0.8682109217500976
RMSE: 1792.2945983769719, R2: 0.7865359252839167
RMSE: 1424.6192554259505, R2: 0.7460260337789031
RMSE: 2817.838976392785, R2: 0.6213272720703027
RMSE: 129

,ORDER_MONTH,ORDER_QTY,PRODUCT_ID
0,2024-08-31,113385.922696,Product_001
1,2024-09-30,113459.197127,Product_001
2,2024-10-31,113520.021482,Product_001
3,2024-11-30,113568.589875,Product_001
4,2024-12-31,113605.095741,Product_001
...,...,...,...
1095,2026-04-30,26458.347300,Product_100
1096,2026-05-31,26410.057419,Product_100
1097,2026-06-30,26361.867746,Product_100
1098,2026-07-31,26312.770305,Product_100


In [36]:
# Let's convert 'ORDER_MONTH' to datetime and proceed with forecasting for each 'PRODUCT_ID'
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt


from fosforml.model_manager.snowflakesession import get_session
my_session = get_session()
 
table_name = '"AGG_QTY_MASTER_DATA"'
 
sf_df = my_session.sql("select * from {}".format(table_name))
df = sf_df.to_pandas()



df['ORDER_MONTH'] = pd.to_datetime(df['ORDER_MONTH'])

print(f"{df['ORDER_MONTH']}")

# Sort the DataFrame by 'ORDER_MONTH' to maintain the time series order
df = df.sort_values(by=['PRODUCT_ID', 'ORDER_MONTH'])

# Function to create multiple lag features for ORDER_QTY
def create_lags(df, target_column, num_lags):
    for lag in range(1, num_lags + 1):
        df[f'Lag_{lag}'] = df[target_column].shift(lag)
    return df

# Define a function that applies the prediction process for each product, including RMSE and R² calculation
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

def forecast_for_product(product_df, num_lags=6, forecast_periods=25):
    # Create lag features for ORDER_QTY
    product_df = create_lags(product_df, 'ORDER_QTY', num_lags)
    
    # Drop rows with NaN values (due to shifting from lag features)
    product_df = product_df.dropna()

    # Convert 'ORDER_MONTH' to an ordinal number to use as a feature
    product_df['ORDER_MONTH_ORDINAL'] = product_df['ORDER_MONTH'].apply(lambda x: x.toordinal())

    # One-hot encode 'PART_CODE'
    part_code_encoded = pd.get_dummies(product_df['PART_CODE'], prefix='PART_CODE')
    
    # Add the encoded 'PART_CODE' columns to the product_df
    product_df = pd.concat([product_df, part_code_encoded], axis=1)

    # Define the features (Lag_1, Lag_2, ..., Lag_n, UNIT_PRICE, LEAD_TIME_IN_WEEKS, ORDER_MONTH_ORDINAL, PART_CODE_encoded_columns)
    lag_columns = [f'Lag_{lag}' for lag in range(1, num_lags + 1)]
    part_code_columns = list(part_code_encoded.columns)
    feature_columns = lag_columns + ['UNIT_PRICE', 'LEAD_TIME_IN_WEEKS', 'ORDER_MONTH_ORDINAL'] + part_code_columns

    # Define X (features) and y (target)
    X = product_df[feature_columns]
    y = product_df['ORDER_QTY']

    # Split the data into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

    # Initialize the Linear Regression model
    model = LinearRegression()

    # Fit the model to the training data
    model.fit(X_train, y_train)

    # Make predictions on the test data
    y_pred = model.predict(X_test)

    # Calculate RMSE
    rmse = mean_squared_error(y_test, y_pred, squared=False)

    # Calculate R²
    r2 = r2_score(y_test, y_pred)

    # Forecast for the next 25 months
    # Create a new DataFrame for future months (last day of the month)
    future_months = pd.date_range(product_df['ORDER_MONTH'].max(), periods=forecast_periods, freq='M')

    # Initialize the last known lag values, UNIT_PRICE, LEAD_TIME_IN_WEEKS, and ORDER_MONTH_ORDINAL
    last_lags = list(product_df[lag_columns].iloc[-1])
    last_unit_price = product_df['UNIT_PRICE'].iloc[-1]
    last_lead_time = product_df['LEAD_TIME_IN_WEEKS'].iloc[-1]
    last_order_month_ordinal = product_df['ORDER_MONTH_ORDINAL'].iloc[-1]
    last_part_code_encoded = product_df[part_code_columns].iloc[-1].values  # Part code one-hot encoding

    # Create an empty list to store predictions
    future_preds = []

    for i in range(forecast_periods):
        # Increment the ORDER_MONTH_ORDINAL for the next future month
        last_order_month_ordinal += 30  # Assuming average month is ~30 days

        # Prepare input features for the next prediction, including lag values, UNIT_PRICE, LEAD_TIME_IN_WEEKS, ORDER_MONTH_ORDINAL, and PART_CODE_encoded columns
        future_X = np.array([last_lags + [last_unit_price, last_lead_time, last_order_month_ordinal] + list(last_part_code_encoded)])

        # Predict the next ORDER_QTY
        future_pred = model.predict(future_X)[0]

        # Append the prediction to the list
        future_preds.append(future_pred)

        # Update lag values for the next iteration
        last_lags = [future_pred] + last_lags[:-1]  # Shift the lags with the new prediction

    # Create a DataFrame for this product's future ORDER_QTY predictions
    print(f"RMSE: {rmse}, R2: {r2}")
    future_forecast_df = pd.DataFrame({
        'ORDER_MONTH': future_months,
        'Predicted_ORDER_QTY': future_preds,
        'PRODUCT_ID': product_df['PRODUCT_ID'].iloc[0],  # Add the PRODUCT_ID to each row
   #     'RMSE': rmse,  # Include the RMSE value for this product
    #    'R2': r2       # Include the R² value for this product
    })

    return future_forecast_df

# Initialize an empty DataFrame to store all product forecasts
all_products_forecast_df = pd.DataFrame()

# Loop over each product group by 'PRODUCT_ID'
for product_id, product_df in df.groupby('PRODUCT_ID'):
    # Apply the forecast function for each product
    product_forecast_df = forecast_for_product(product_df)
    
    # Append the forecast for this product to the overall forecast DataFrame
    all_products_forecast_df = pd.concat([all_products_forecast_df, product_forecast_df], ignore_index=True)

# Rename the columns
all_products_forecast_df.rename(columns={'Predicted_ORDER_QTY': 'ORDER_QTY'}, inplace=True)

all_products_forecast_df

0       2019-01-01
1       2019-02-01
2       2019-03-01
3       2019-04-01
4       2019-05-01
           ...    
38755   2024-04-01
38756   2024-05-01
38757   2024-06-01
38758   2024-07-01
38759   2024-08-01
Name: ORDER_MONTH, Length: 38760, dtype: datetime64[ns]
RMSE: 3684.96951633217, R2: 0.9435835281438437
RMSE: 2837.883018438868, R2: 0.9266389249399793
RMSE: 899.8098559040291, R2: 0.9719617224702137
RMSE: 1603.7804895497497, R2: 0.9684794861623811
RMSE: 1097.0202249228275, R2: 0.9614773107556353
RMSE: 3173.1837001856456, R2: 0.9298099207590432
RMSE: 3245.025410685687, R2: 0.9217855277042188
RMSE: 2276.2962326680795, R2: 0.9655836058152807
RMSE: 3039.2300116824968, R2: 0.9677923915510038
RMSE: 2178.108976108043, R2: 0.9460986531243109
RMSE: 1011.3765852915262, R2: 0.9162471202501593
RMSE: 2004.9045769473119, R2: 0.8682821907886304
RMSE: 1794.1730973214317, R2: 0.7860882285953102
RMSE: 1425.4789894483565, R2: 0.7457194031559515
RMSE: 2817.867164387482, R2: 0.6213196959963507
RMSE: 1

,ORDER_MONTH,ORDER_QTY,PRODUCT_ID
0,2024-08-31,113400.796921,Product_001
1,2024-09-30,113488.757675,Product_001
2,2024-10-31,113564.082863,Product_001
3,2024-11-30,113626.970200,Product_001
4,2024-12-31,113677.613678,Product_001
...,...,...,...
1095,2026-04-30,26458.347300,Product_100
1096,2026-05-31,26410.057419,Product_100
1097,2026-06-30,26361.867746,Product_100
1098,2026-07-31,26312.770305,Product_100


In [41]:
# Let's convert 'ORDER_MONTH' to datetime and proceed with forecasting for each 'PRODUCT_ID'
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt


from fosforml.model_manager.snowflakesession import get_session
my_session = get_session()
 
table_name = '"AGG_QTY_MASTER_DATA"'
 
sf_df = my_session.sql("select * from {}".format(table_name))
df = sf_df.to_pandas()



df['ORDER_MONTH'] = pd.to_datetime(df['ORDER_MONTH'])

print(f"{df['ORDER_MONTH']}")

# Sort the DataFrame by 'ORDER_MONTH' to maintain the time series order
df = df.sort_values(by=['PRODUCT_ID', 'ORDER_MONTH'])

# Function to create multiple lag features for ORDER_QTY
def create_lags(df, target_column, num_lags):
    for lag in range(1, num_lags + 1):
        df[f'Lag_{lag}'] = df[target_column].shift(lag)
    return df

# Define a function that applies the prediction process for each product, including RMSE and R² calculation
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

def forecast_for_product(product_df, num_lags=6, forecast_periods=25):
    # Create lag features for ORDER_QTY
    product_df = create_lags(product_df, 'ORDER_QTY', num_lags)
    
    # Drop rows with NaN values (due to shifting from lag features)
    product_df = product_df.dropna()

    # Convert 'ORDER_MONTH' to an ordinal number to use as a feature
    product_df['ORDER_MONTH_ORDINAL'] = product_df['ORDER_MONTH'].apply(lambda x: x.toordinal())

    # One-hot encode 'PART_CODE'
    part_code_encoded = pd.get_dummies(product_df['PART_CODE'], prefix='PART_CODE')
    
    # Add the encoded 'PART_CODE' columns to the product_df
    product_df = pd.concat([product_df, part_code_encoded], axis=1)

    # Define the features (Lag_1, Lag_2, ..., Lag_n, UNIT_PRICE, LEAD_TIME_IN_WEEKS, ORDER_MONTH_ORDINAL, PART_CODE_encoded_columns)
    lag_columns = [f'Lag_{lag}' for lag in range(1, num_lags + 1)]
    part_code_columns = list(part_code_encoded.columns)
    feature_columns = lag_columns + ['UNIT_PRICE', 'LEAD_TIME_IN_WEEKS', 'ORDER_MONTH_ORDINAL'] + part_code_columns

    # Define X (features) and y (target)
    X = product_df[feature_columns]
    y = product_df['ORDER_QTY']

    # Split the data into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

    # Initialize the Linear Regression model
    model = LinearRegression()

    # Fit the model to the training data
    model.fit(X_train, y_train)

    # Make predictions on the test data
    y_pred = model.predict(X_test)

    # Calculate RMSE
    rmse = mean_squared_error(y_test, y_pred, squared=False)

    # Calculate R²
    r2 = r2_score(y_test, y_pred)

    # Forecast for the next 25 months
    # Create a new DataFrame for future months (start of the month)
    future_months = pd.date_range(product_df['ORDER_MONTH'].max(), periods=forecast_periods, freq='MS')

    # Initialize the last known lag values, UNIT_PRICE, LEAD_TIME_IN_WEEKS, and ORDER_MONTH_ORDINAL
    last_lags = list(product_df[lag_columns].iloc[-1])
    last_unit_price = product_df['UNIT_PRICE'].iloc[-1]
    last_lead_time = product_df['LEAD_TIME_IN_WEEKS'].iloc[-1]
    last_order_month_ordinal = product_df['ORDER_MONTH_ORDINAL'].iloc[-1]
    last_part_code_encoded = product_df[part_code_columns].iloc[-1].values  # Part code one-hot encoding

    # Create an empty list to store predictions
    future_preds = []

    for i in range(forecast_periods):
        # Increment the ORDER_MONTH_ORDINAL for the next future month
        last_order_month_ordinal += 30  # Assuming average month is ~30 days

        # Prepare input features for the next prediction, including lag values, UNIT_PRICE, LEAD_TIME_IN_WEEKS, ORDER_MONTH_ORDINAL, and PART_CODE_encoded columns
        future_X = np.array([last_lags + [last_unit_price, last_lead_time, last_order_month_ordinal] + list(last_part_code_encoded)])

        # Predict the next ORDER_QTY
        future_pred = model.predict(future_X)[0]

        # Append the prediction to the list
        future_preds.append(future_pred)

        # Update lag values for the next iteration
        last_lags = [future_pred] + last_lags[:-1]  # Shift the lags with the new prediction

    # Format future_months as 'YYYY-MMM' (e.g., '2024-Aug')
    future_months_formatted = future_months.strftime('%Y-%b')
    print(f"RMSE: {rmse}, R2: {r2}")
    # Create a DataFrame for this product's future ORDER_QTY predictions
    future_forecast_df = pd.DataFrame({
        'ORDER_MONTH': future_months_formatted,
        'Predicted_ORDER_QTY': future_preds,
        'PRODUCT_ID': product_df['PRODUCT_ID'].iloc[0],  # Add the PRODUCT_ID to each row
        #'RMSE': rmse,  # Include the RMSE value for this product
        #'R2': r2       # Include the R² value for this product
    })

    return future_forecast_df

# Initialize an empty DataFrame to store all product forecasts
all_products_forecast_df = pd.DataFrame()

# Loop over each product group by 'PRODUCT_ID'
for product_id, product_df in df.groupby('PRODUCT_ID'):
    # Apply the forecast function for each product
    product_forecast_df = forecast_for_product(product_df)
    
    # Append the forecast for this product to the overall forecast DataFrame
    all_products_forecast_df = pd.concat([all_products_forecast_df, product_forecast_df], ignore_index=True)

# Rename the columns
all_products_forecast_df.rename(columns={'Predicted_ORDER_QTY': 'ORDER_QTY'}, inplace=True)

all_products_forecast_df

0       2019-01-01
1       2019-02-01
2       2019-03-01
3       2019-04-01
4       2019-05-01
           ...    
38755   2024-04-01
38756   2024-05-01
38757   2024-06-01
38758   2024-07-01
38759   2024-08-01
Name: ORDER_MONTH, Length: 38760, dtype: datetime64[ns]
RMSE: 3684.96951633217, R2: 0.9435835281438437
RMSE: 2837.883018438868, R2: 0.9266389249399793
RMSE: 899.8098559040291, R2: 0.9719617224702137
RMSE: 1603.7804895497497, R2: 0.9684794861623811
RMSE: 1097.0202249228275, R2: 0.9614773107556353
RMSE: 3173.1837001856456, R2: 0.9298099207590432
RMSE: 3245.025410685687, R2: 0.9217855277042188
RMSE: 2276.2962326680795, R2: 0.9655836058152807
RMSE: 3039.2300116824968, R2: 0.9677923915510038
RMSE: 2178.108976108043, R2: 0.9460986531243109
RMSE: 1011.3765852915262, R2: 0.9162471202501593
RMSE: 2004.9045769473119, R2: 0.8682821907886304
RMSE: 1794.1730973214317, R2: 0.7860882285953102
RMSE: 1425.4789894483565, R2: 0.7457194031559515
RMSE: 2817.867164387482, R2: 0.6213196959963507
RMSE: 1

,ORDER_MONTH,ORDER_QTY,PRODUCT_ID
0,2024-Aug,113400.796921,Product_001
1,2024-Sep,113488.757675,Product_001
2,2024-Oct,113564.082863,Product_001
3,2024-Nov,113626.970200,Product_001
4,2024-Dec,113677.613678,Product_001
...,...,...,...
1095,2026-Apr,26458.347300,Product_100
1096,2026-May,26410.057419,Product_100
1097,2026-Jun,26361.867746,Product_100
1098,2026-Jul,26312.770305,Product_100


In [44]:
sf_df = my_session.createDataFrame(all_products_forecast_df)
sf_df.write.mode("overwrite").save_as_table("ORDER_FORECAST")
my_session.table("ORDER_FORECAST").show()

-----------------------------------------------------
|"ORDER_MONTH"  |"ORDER_QTY"         |"PRODUCT_ID"  |
-----------------------------------------------------
|2024-Aug       |113400.79692077637  |Product_001   |
|2024-Sep       |113488.7576751709   |Product_001   |
|2024-Oct       |113564.082862854    |Product_001   |
|2024-Nov       |113626.97019958496  |Product_001   |
|2024-Dec       |113677.61367797852  |Product_001   |
|2025-Jan       |113716.20524597168  |Product_001   |
|2025-Feb       |113741.36532592773  |Product_001   |
|2025-Mar       |113753.50351715088  |Product_001   |
|2025-Apr       |113753.01989746094  |Product_001   |
|2025-May       |113740.30521392822  |Product_001   |
-----------------------------------------------------

